In [89]:
import sys
import time
from confluent_kafka import Consumer, KafkaError
from json import loads
from kafka import KafkaConsumer
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

In [90]:
# Kafka broker address
bootstrap_servers = 'localhost:9093'
 
# Kafka topic to consume messages from
kafka_topic = 'QM_data_SPO'
 
# Consumer group ID
group_id = 'group1'
 
# InfluxDB connection details
influxdb_host = 'localhost'
influxdb_port = 8086
influxdb_username = 'shams'
influxdb_password = 'shams123'
bucket = "Monitoring_SPO2"
organization = "OstProject"
influxdb_token = "FlRUvysHQ7GXF3N61Ii5VmtXn1xd-4-HYWxZRMSHmo8JJ_pODFGEkiPcBnYrET-blDswyQqheC0eRN1RA_ZJBA=="

# Create Kafka consumer configuration
consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': group_id,
    'auto.offset.reset': 'earliest'
}

In [91]:
# Variables for current and lowest SPO2
lowest_spo2 = float('inf')

In [92]:
with InfluxDBClient(url="http://localhost:8086", token=influxdb_token, org=organization,bucket=bucket) as client:
        write_api = client.write_api(write_options=SYNCHRONOUS)
        consumer = KafkaConsumer(kafka_topic, bootstrap_servers=['localhost:9093'],auto_offset_reset='earliest', api_version=(0,10),enable_auto_commit=True,value_deserializer=lambda x: loads(x.decode('utf-8')))
        print("Connection established")
        i=0
        for message in consumer:
            dict_values = message.value

            # Extract relevant values
            timestamp = dict_values.get('Timestamp', "")
            spo = float(dict_values.get('SPO2', ""))

            # Create an InfluxDB point
            point = Point("SPO2_Monitoring") \
                .field("Timestamp", timestamp) \
                .field("SPO2", spo) \
                .time(datetime.utcnow(), WritePrecision.NS)

            write_api.write(bucket, organization, point)
            i += 1
            print(point)

            current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

            if spo < lowest_spo2:
                lowest_spo2 = spo
                print(f"Lowest SBP Observed: {lowest_spo2} at {current_time}")

                lowest_spo2_point = Point("Lowest_SPO2_Monitoring") \
                    .field("lowest_spo2", lowest_spo2) \
                    .field("lowest_spo2_timestamp", current_time) \
                    .time(datetime.utcnow(), WritePrecision.NS)
                write_api.write(bucket, organization, lowest_spo2_point)
            else:
                 write_api.write(bucket, organization, point=lowest_spo2_point)
            print("Message sent to InfluxDB", i)
            time.sleep(60)

Connection established
SPO2_Monitoring SPO2=94.3,Timestamp="2020-10-24 21:37" 1702927504771451000
Lowest SBP Observed: 94.3 at 2023-12-18 19:25:04
Message sent to InfluxDB 1
SPO2_Monitoring SPO2=94.1,Timestamp="2020-10-24 21:38" 1702927564848576000
Lowest SBP Observed: 94.1 at 2023-12-18 19:26:04
Message sent to InfluxDB 2
SPO2_Monitoring SPO2=94,Timestamp="2020-10-24 21:39" 1702927624947231000
Lowest SBP Observed: 94.0 at 2023-12-18 19:27:05
Message sent to InfluxDB 3
SPO2_Monitoring SPO2=93.7,Timestamp="2020-10-24 21:40" 1702927685066283000
Lowest SBP Observed: 93.7 at 2023-12-18 19:28:05
Message sent to InfluxDB 4
SPO2_Monitoring SPO2=93.7,Timestamp="2020-10-24 21:41" 1702927745187748000
Message sent to InfluxDB 5
SPO2_Monitoring SPO2=93.9,Timestamp="2020-10-24 21:42" 1702927805259548000
Message sent to InfluxDB 6
SPO2_Monitoring SPO2=94.5,Timestamp="2020-10-24 21:43" 1702927865312312000
Message sent to InfluxDB 7
SPO2_Monitoring SPO2=92.8,Timestamp="2020-10-24 21:44" 17029279253733